In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd 

headers = {'User Agent':'Mozilla/5.0'}

players = [
    'kyrie-irving', 'derrick-williams-2', 'tristan-thompson-2', 'brandon-knight', 'kemba-walker',
    'jimmer-fredette', 'klay-thompson','alec-burks', 'markieff-morris', 'marcus-morris', 'kawhi-leonard', 
    'nikola-vucevic', 'iman-shumpert', 'chris-singleton','tobias-harris', 'nolan-smith', 'kenneth-faried', 
    'reggie-jackson', 'marshon-brooks', 'jordan-hamilton', 'jajuan-johnson', 'norris-cole', 'cory-joseph', 
    'jimmy-butler', 'justin-harper', 'kyle-singler', 'shelvin-mack', 'tyler-honeycutt', 'trey-thompkins',
    'chandler-parsons', 'jon-leuer', 'darius-morris', 'malcolm-lee', 'charles-jenkins', 'josh-harrellson', 
    'travis-leslie', 'keith-benson', 'josh-shelby', 'lavoy-allen', 'jon-diebler', 'vernon-macklin', 
    'deandre-liggins', 'etwaun-moore', 'isaiah-thomas']   

player_stats = []
playerlist = []
year_list = []
age_list = []

for player in players:
    try:
        if player[-1].isdigit():
            url = f'https://www.sports-reference.com/cbb/players/{player}.html'
        else:
            url = f'https://www.sports-reference.com/cbb/players/{player}-1.html'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        header = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')]
        rows = soup.findAll('tr')
        player_stats.append([td.getText() for td in soup.find('tr', id ='players_per_game.2011')])
        table = pd.DataFrame(player_stats, columns = header) 
        for i in player_stats:
            if i[14] == '':
                i[14] = str(0.0)
        
        player = player.replace('-',' ').title()
        if player[-1].isdigit():
            player = player[:-2]
        if player[-2:] == 'jr':
            player = player.replace('jr', ' Jr')
        playerlist.append(player)
        
    except:
        continue

    try:
        url = f'https://www.google.com/search?q={player}+basketball+birth+date'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        age = (soup.findAll('div', class_ = 'BNeawe iBp4i AP7Wnd')[0])
        for x in age:
            x = x.text
        x = x.replace(',','').split()
        months = {
            'Janurary': '1',
            'February': '2',
            'March': '3',
            'April': '4',
            'May': '5',
            'June': '6',
            'July': '7',
            'August': '8',
            'September': '9',
            'October': '10',
            'November': '11',
            'December': '12'
        } 
        birth_date = months[x[0]]+ '-' + x[1] + '-' + x[2]
        draft_date = '7-1-2011'
        import datetime 
        birth_date_final = datetime.datetime.strptime(birth_date, '%m-%d-%Y')  
        draft_date_final = datetime.datetime.strptime(draft_date, '%m-%d-%Y')

        age_on_draft_night = draft_date_final - birth_date_final
        age = round(age_on_draft_night.days/365, 1)
        age_list.append(age)
        year = round(((1-(age/35))*2.33), 2)
        year_list.append(year)
        
    except:
        age_list.append("No data")
        year_list.append(0)
    
table.insert(1, "Age", age_list)
table.insert(2, "Year", year_list)
table.insert(0, "Name", playerlist)

table['MP'] = table['MP'].astype(float)
table['PTS'] = table['PTS'].astype(float)
table['AST'] = table['AST'].astype(float)
table['TRB'] = table['TRB'].astype(float)
table['BLK'] = table['BLK'].astype(float)
table['STL'] = table['STL'].astype(float)
table['3P'] = table['3P'].astype(float)
table['3PA'] = table['3PA'].astype(float)
table['TOV'] = table['TOV'].astype(float)
table['SOS'] = table['SOS'].astype(float)
table['PF'] = table['PF'].astype(float)
table['FT%'] = table['FT%'].astype(float)
table['3P%'] = table['3P%'].astype(float)

table["Player Grade"] = ((table['PTS']) + (table['TRB']*1.25) + (table['AST']*2) +
(table['BLK']*2) + (table['STL']*3) + (table['3P']*2) + (table['3PA']) + (table['SOS']/2)) * table['Year']

table["Player Grade"] = table["Player Grade"]*1.75
table["Player Grade"] = (round(table["Player Grade"], 1))
table["Player Grade"]= table["Player Grade"].astype(float)
                        
table = table.sort_values(by= "Player Grade", ascending=False)
table.insert(5, 'SoS', table['SOS'])
table.reset_index(drop = True, inplace=True)
import numpy as np
table.index = np.arange(1, len(table)+1)
del table['SOS']
del table['Conf']
del table['ORB']
del table['DRB']
del table['GS']
del table['FG']
del table['FGA']
del table['FT']
del table['FTA']
del table['2P']
del table['2PA']
del table['2P%']
del table['Year']
del table['TOV']
del table['PF']
table

,Name,Season,Age,School,SoS,G,MP,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,PTS,,Player Grade
1,Kemba Walker,2010-11,21.2,UConn,10.44,41,37.6,.428,1.8,5.5,0.330,0.819,5.4,4.5,1.9,0.2,23.5,,96.1
2,Jimmer Fredette,2010-11,22.4,BYU,5.76,37,35.8,.452,3.4,8.5,0.396,0.894,3.4,4.3,1.3,0.0,28.9,,93.8
3,Klay Thompson,2010-11,21.4,Washington State,6.81,34,34.7,.436,2.9,7.2,0.398,0.838,5.2,3.7,1.6,0.9,21.6,,93.2
4,Brandon Knight,2010-11,19.6,Kentucky,8.91,38,35.9,.423,2.3,6.1,0.377,0.795,4.0,4.2,0.7,0.2,17.3,,87.2
5,Kyrie Irving,2010-11,19.3,Duke,8.32,11,27.5,.529,1.6,3.5,0.462,0.901,3.4,4.3,1.5,0.5,17.5,,85.8
6,Iman Shumpert,2010-11,21.0,Georgia Tech,5.68,31,32.0,.406,1.4,4.9,0.278,0.806,5.9,3.5,2.7,0.2,17.3,,82.5
7,Kawhi Leonard,2010-11,20.0,San Diego State,6.36,36,32.6,.444,0.7,2.4,0.291,0.759,10.6,2.5,1.4,0.6,15.5,,80.7
8,Reggie Jackson,2010-11,21.2,Boston College,7.31,34,34.1,.503,2.1,5.0,0.420,0.796,4.3,4.5,1.1,0.5,18.2,,80.1
9,Alec Burks,2010-11,20.0,Colorado,5.61,38,31.4,.469,0.7,2.5,0.292,0.825,6.5,2.9,1.1,0.3,20.5,,78.8
10,Norris Cole,2010-11,22.7,Cleveland State,1.74,36,35.7,.439,1.4,4.1,0.342,0.853,5.8,5.3,2.2,0.1,21.7,,77.7
